In [ ]:
!pip install openpyxl
!pip install xlsxwriter

In [2]:
import pandas as pd
import openpyxl
import xlsxwriter
from datetime import date



In [3]:
input_df = pd.read_excel('Book1.xlsx',index_col=False)
input_df

,CamName,Asin,DailyBudget,ProductName,Bid
0,ABC1,SA001,1000.0,Example,0.7
1,NaN,SA002,NaN,NaN,NaN


In [4]:
# Read Information from excel file
def read_information_setup(excel_input=''): 
    input_df = pd.read_excel(excel_input,index_col=False)
    # cam_name.remove('nan')
    cam_list = [x for x in input_df['CamName'].to_list() if str(x) != 'nan']
    asin_list = [x for x in input_df['Asin'].to_list() if str(x) != 'nan']
    budget_list = [x for x in input_df['DailyBudget'].to_list() if str(x) != 'nan']
    productName_list = [x for x in input_df['ProductName'].to_list() if str(x) != 'nan']
    bid_list = [x for x in input_df['Bid'].to_list() if str(x) != 'nan']
    return  cam_list[0],asin_list,budget_list[0],productName_list[0],bid_list[0]

In [5]:
x1,x2,x3,x4,x5 = read_information_setup('Book1.xlsx')

In [6]:
print(x1)
print(x2)
print(x3)
print(x4)
print(x5)

ABC1
['SA001', 'SA002']
1000.0
Example
0.7


In [15]:
# Create Setup DataFrame

def create_single_row(columns:list,cam_id):
    single_row_dictionary = {}
    for item in columns:
        if item == 'Product':
            single_row_dictionary[item] = 'Sponsored Products'
        elif item == 'Operation':
            single_row_dictionary[item] = 'Create'
        elif item == 'Campaign Id':
            single_row_dictionary[item] = cam_id
        elif item == 'State':
            single_row_dictionary[item] = 'enabled'
        else:
            single_row_dictionary[item] = None
    return single_row_dictionary

def createDataFrame_asin(cam_id,asin_list:list,budget,productName,bid,billing_strategy):
    columns=['Product','Entity','Operation','Campaign Id','Ad Group Id','Portfolio Id',
                           'Ad Id','Keyword Id','Product Targeting Id','Campaign Name','Ad Group Name',
                           'Start Date','End Date','Targeting Type','State','Daily Budget','sku','asin',
                           'Ad Group Default Bid','Bid','Keyword Text','Match Type','Bidding Strategy',
                           'Placement','Percentage','Product Targeting Expression']
    df = pd.DataFrame(columns=columns)

# 5 dong co dinh
    for i in range (0,5):
        df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
        
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campagin Name'] = cam_id
    df.iloc[0]['Targeting Type'] = 'Manual'
    df.iloc[0]['Start Date'] = date.today().strftime("%Y%m%d")
    df.iloc[0]['Daily Budget'] = budget
    df.iloc[0]['Bidding Strategy'] = billing_strategy

    df.iloc[1]['Entity'] = 'Bidding Adjustment'
    df.iloc[1]['Placement'] = 'placementTop'
    df.iloc[1]['Percentage'] = '0%'

    df.iloc[2]['Entity'] = 'Bidding Adjustment'
    df.iloc[2]['Placement'] = 'placementProductPage'
    df.iloc[2]['Percentage'] = '0%'

    df.iloc[3]['Entity'] = 'Ad group'
    df.iloc[3]['Ad Group Id'] = cam_id
    df.iloc[3]['Ad Group Name'] = cam_id
    df.iloc[3]['Ad Group Default Bid'] = bid
    
    df.iloc[4]['Entity'] = 'Product ad'
    df.iloc[4]['Ad Group Id'] = cam_id
    df.iloc[4]['sku'] = productName

# Optional Asin - so dong Asin  
    for item in asin_list:
        asin_row = create_single_row(columns=columns,cam_id=cam_id)
        asin_row['Entity'] = 'Product targeting'
        asin_row['Ad Group Id'] = cam_id
        asin_row['Product Targeting Expression'] = '''asin="''' +str(item)+ '''"'''
        df = df.append(asin_row,ignore_index=True)
    
    return df

In [16]:
# Create DataFrame - Case Keyword
def createDataFrame_keyword(cam_id,keyword_list:list,budget,productName,bid,billing_strategy,portfolio_id,match_type):
    columns=['Product','Entity','Operation','Campaign Id','Ad Group Id','Portfolio Id',
                           'Ad Id','Keyword Id','Product Targeting Id','Campaign Name','Ad Group Name',
                           'Start Date','End Date','Targeting Type','State','Daily Budget','sku','asin',
                           'Ad Group Default Bid','Bid','Keyword Text','Match Type','Bidding Strategy',
                           'Placement','Percentage','Product Targeting Expression']
    df = pd.DataFrame(columns=columns)

# 5 dong co dinh
    for i in range (0,5):
        df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
        
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campagin Name'] = cam_id
    df.iloc[0]['Targeting Type'] = 'Manual'
    df.iloc[0]['Start Date'] = date.today().strftime("%Y%m%d")
    df.iloc[0]['Daily Budget'] = budget
    df.iloc[0]['Bidding Strategy'] = billing_strategy
    df.iloc[0]['Portfolio Id'] = portfolio_id

    df.iloc[1]['Entity'] = 'Bidding Adjustment'
    df.iloc[1]['Placement'] = 'placementTop'
    df.iloc[1]['Percentage'] = '0%'

    df.iloc[2]['Entity'] = 'Bidding Adjustment'
    df.iloc[2]['Placement'] = 'placementProductPage'
    df.iloc[2]['Percentage'] = '0%'

    df.iloc[3]['Entity'] = 'Ad group'
    df.iloc[3]['Ad Group Id'] = cam_id
    df.iloc[3]['Ad Group Name'] = cam_id
    df.iloc[3]['Ad Group Default Bid'] = bid
    
    df.iloc[4]['Entity'] = 'Product ad'
    df.iloc[4]['Ad Group Id'] = cam_id
    df.iloc[4]['sku'] = productName
    df.iloc[4]['Bid'] = bid

# Optional Product Targetting- so dong Keyword
    for item in keyword_list:
        keyword_row = create_single_row(columns=columns,cam_id=cam_id)
        keyword_row['Entity'] = 'Product targeting'
        keyword_row['Ad Group Id'] = cam_id
        keyword_row['Keyword Text'] = str(item)
        keyword_row['Match Type'] = match_type
        df = df.append(keyword_row,ignore_index=True)
    
    return df

In [13]:
# output_df = createDataFrame_asin(x1,x2,x3,x4,x5)
# output_df"
output_df = createDataFrame_keyword(cam_id='ASD123456',keyword_list=['Khai1','Khai2'],budget=2.9,productName='Newspaper',bid=0.6,billing_strategy='Fixed_bid',portfolio_id='5555555555',match_type='Exact')
output_df

,Product,Entity,Operation,Campaign Id,Ad Group Id,Portfolio Id,Ad Id,Keyword Id,Product Targeting Id,Campaign Name,...,sku,asin,Ad Group Default Bid,Bid,Keyword Text,Match Type,Bidding Strategy,Placement,Percentage,Product Targeting Expression
0,Sponsored Products,Campaign,Create,ABC1,None,5555555555,None,None,None,None,...,None,None,None,None,None,None,Fixed_bid,None,None,None
1,Sponsored Products,Bidding Adjustment,Create,ABC1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementTop,0%,None
2,Sponsored Products,Bidding Adjustment,Create,ABC1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementProd,0%,None
3,Sponsored Products,Ad group,Create,ABC1,ABC1,None,None,None,None,None,...,None,None,0.6,None,None,None,None,None,None,None
4,Sponsored Products,Product ad,Create,ABC1,ABC1,None,None,None,None,None,...,Newspaper,None,None,0.6,None,None,None,None,None,None
5,Sponsored Products,Product targeting,Create,ABC1,ABC1,None,None,None,None,None,...,None,None,None,None,Khai1,Exact,None,None,None,None
6,Sponsored Products,Product targeting,Create,ABC1,ABC1,None,None,None,None,None,...,None,None,None,None,Khai2,Exact,None,None,None,None


In [14]:
# df.to_excel('myexcel.xlsx',sheet_name='Sheet1',engine='xlsxwriter')
writer = pd.ExcelWriter("dataframe2.xlsx", engine='xlsxwriter')
output_df.to_excel(writer,sheet_name = 'sheet4', index=False)
writer.save() 